In [495]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, TimeoutError, LocalCluster
import pandas as pd
import hvplot.dask
import re


#code found on stack exchange
#makes a new client if one is not running, runs 4 workers
try:
    client = Client('tcp://localhost:8787', timeout='2s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=4, ip='localhost')
    client = Client(cluster)
#

client


Task exception was never retrieved
future: <Task finished name='Task-60126' coro=<Client._gather.<locals>.wait() done, defined at /Users/derek/opt/anaconda3/lib/python3.9/site-packages/distributed/client.py:2002> exception=AllExit()>
Traceback (most recent call last):
  File "/Users/derek/opt/anaconda3/lib/python3.9/site-packages/distributed/client.py", line 2011, in wait
    raise AllExit()
distributed.client.AllExit


<Client: 'tcp://127.0.0.1:8787' processes=4 threads=8, memory=16.00 GiB>

In [496]:
keyWords = [
    'mental health', 
    'depression',
    'depressed', 
    'anxiety',
    'anxious', 
    'ADHD', 
    'attention deficit',
    'OCD',
    'obsessive-compulsive disorder',
    'PTSD',
    'posttraumatic stress disorder',
    'trauma',
    'bipolar',
    'autism',
    'schizophrenia',
    'eating disorder',
    'anorexia',
    'bulimia',
    'psychosis',
    'psychologist',
    'psychotherapist',
    'psychiatrist',
    'psychotherapy',
    'depressant'
]

In [497]:
#read and print the dataframes

#contains tweets (and time of the tweet) that contain mental health key word
#polled ever 1 minute
tweetdf = dd.read_csv(urlpath='mental_health_tweets.csv', sep='\t' )

#contains counts of tweets for every hour
countdf = dd.read_csv(urlpath='mental_health_tweet_count.csv', sep='\t')

print(tweetdf)

print(countdf)

Dask DataFrame Structure:
                tweet    time
npartitions=1                
               object  object
                  ...     ...
Dask Name: read-csv, 1 tasks
Dask DataFrame Structure:
              timestamp  count
npartitions=1                 
                 object  int64
                    ...    ...
Dask Name: read-csv, 1 tasks


In [498]:

#specify column types
tweetdf['time'] = dd.to_datetime(tweetdf['time'])
countdf['timestamp'] = dd.to_datetime(countdf['timestamp'])

#separate the components of the time for grouping
tweetdf['year'] = tweetdf['time'].dt.year
tweetdf['month'] = tweetdf['time'].dt.month
tweetdf['day'] = tweetdf['time'].dt.day
tweetdf['hour'] = tweetdf['time'].dt.hour
tweetdf['minute'] = tweetdf['time'].dt.minute

countdf['year'] = countdf['timestamp'].dt.year
countdf['month'] = countdf['timestamp'].dt.month
countdf['day'] = countdf['timestamp'].dt.day
countdf['hour'] = countdf['timestamp'].dt.hour

In [499]:
#sort the dataframes based on ascening tme
countdf = countdf.sort_values(ascending=True, by=['month', 'day', 'hour'])
tweetdf = tweetdf.sort_values(ascending=True, by=['month', 'day', 'hour', 'minute'])

#make a display string for the x axis of the graph
countdf['ts_display'] = countdf['month'].astype('str') + '/' + countdf['day'].astype('str') + '\t' + countdf['hour'].astype('str') + ':00'

#TODO MAKE THE GRAPH LOOK BETTER
countdf.hvplot(x='ts_display', y='count').opts(xrotation=90, fontscale=0.5)


:Curve   [ts_display]   (count)

In [500]:
#pre process

def preprocess(text):
    text = text[1]
    text = text.lower()
    return re.sub(string=text, pattern='[^a-z ]', repl='')

tweetdf['text'] = tweetdf['tweet'].map_partitions(preprocess)

In [501]:
#determine key words in tweet
def containsKey(text, key):
    return text.str.contains(key)

for i in keyWords:
    tweetdf[i] = tweetdf['tweet'].map_partitions(containsKey, i)


In [502]:
def keysInTweet(df):
    keys = ''
    for i in keyWords:
        if df[i]:
            keys = keys + ' ' + i
    key = keys.strip()
    return keys

tweetdf['keys'] = tweetdf.apply(keysInTweet, axis=1, meta=('keys', 'object'))

tweetdf = tweetdf.loc[tweetdf['keys'] != '']

In [503]:
counts = tweetdf['keys'].value_counts().to_frame()
counts['count'] = counts['keys']
counts = counts.drop(labels='keys', axis=1)
counts = counts.nlargest(n=30, columns='count')
counts.compute()

,count
trauma,11582
mental health,11272
depression,7026
autism,2865
PTSD,1742
bipolar,1567
depression anxiety,1044
psychiatrist,796
psychologist,765
psychosis,573


In [504]:
counts.hvplot.bar(y='count', use_index=True)

:Bars   [index]   (count)